<a href="https://colab.research.google.com/github/amina04/CNN-lstm/blob/main/CNN_cicids2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd  
import tensorflow as tf

In [3]:
d1=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
d2=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
d3=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Friday-WorkingHours-Morning.pcap_ISCX.csv')
d4=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Monday-WorkingHours.pcap_ISCX.csv')
d5=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
d6=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
d7=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Wednesday-workingHours.pcap_ISCX.csv')
d8=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Tuesday-WorkingHours.pcap_ISCX.csv')


In [4]:
data=pd.concat([d1,d2,d3,d4,d5,d6,d7,d8])
data.shape

(2830743, 79)

In [ ]:
data=pd.concat([d1,d2,d3,d4,d5,d6,d7,d8]).drop_duplicates(keep=False)
data.reset_index(drop=True, inplace = True)

In [ ]:
del d1,d2,d3,d4,d5,d6,d7

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.columns
print(data[' Label'].unique())
len(data[' Label'].unique())

**Data Cleaning**
This chapter contains data cleaning code. We go through the process of renaming columns, removing NaN and non-finite values (-inf, inf) to get the data ready for visualization and model training.

In [ ]:
# Removing whitespaces in column names.
#renommer les colonnes
col_names = [col.replace(' ', '') for col in data.columns]
data.columns = col_names
data.head()

In [ ]:
data.columns
print(data['Label'].unique())
len(data['Label'].unique())

In [ ]:
# Here we can see that 'Label' column contains some wierd characters. 

data["Label"].unique()

In [ ]:
# This next snippet uses regular expressions to replace wierd characters with dunders.

label_names = data['Label'].unique()


import re

label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]

label_names, len(label_names)

In [ ]:
# Replacing 'Label' column values with new readable values.

labels = data['Label'].unique()

for i in range(0,len(label_names)):
    data['Label'] = data['Label'].replace({labels[i] : label_names[i]})  
data['Label'].unique()

In [ ]:
# Checking if there are any NULL values in the dataset.

data.isnull().values.any()

In [ ]:
# Checking which column/s contain NULL values.

[col for col in data if data[col].isnull().values.any()]

In [ ]:
# Checking how many NULL values it this column contains.

data['FlowBytes/s'].isnull().sum()

In [ ]:
# Removing rows that contain NULL values and checking if number of removed rows is equal to the number of null values.

before = data.shape
data.dropna(inplace=True)
after = data.shape
before[0] - after[0]

In [ ]:
data.isnull().any().any()

**Removing non-finite values**

In [ ]:
import numpy as np

In [ ]:
labl = data['Label']
data = data.loc[:, data.columns != 'Label'].astype('float64')

In [ ]:
# Checking if all values are finite.
np.all(np.isfinite(data))

In [ ]:
# Checking what column/s contain non-finite values.

nonfinite = [col for col in data if not np.all(np.isfinite(data[col]))]
nonfinite

In [ ]:
# Checking how many non-finite values each column contains.

finite= np.isfinite(data['FlowBytes/s']).sum()
data.shape[0] - finite

In [ ]:
# Checking how many non-finite values each column contains.

finite= np.isfinite(data['FlowPackets/s']).sum()
data.shape[0] - finite

In [ ]:
# Same as before, since there is a small number of non-finite values we can safely remove them from the dataset
# without spoiling the dataset.

# Replacing infinite values with NaN values.
data = data.replace([np.inf,-np.inf],np.nan)

In [ ]:
# We can see that now we have Nan values again.

np.any(np.isnan(data))

In [ ]:
# Bringing the Labels back into the dataset before deliting Nan rows.

data = data.merge(labl, how='outer', left_index=True, right_index=True)

In [ ]:
# Removing new NaN values.

before = data.shape
data.dropna(inplace=True)
after = data.shape
before[0] - after[0]

In [ ]:
data.shape

**Data preparation**

In [ ]:
# Splitting dataset into features and labels.

labels = data['Label']
features = data.loc[:, data.columns != 'Label'].astype('float64')

In [ ]:
labels.head()
i=0
for row in labels:
  if row!='BENIGN':
    i+=1
print(i)

In [ ]:
# For scaling the data, we use RobustScaler class from sklearn.

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features = scaler.fit_transform(features)


In [ ]:
# Checking if scaling has been succesful.
features[0]

**Label encoding**

In [ ]:
#multiclass
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
LE.fit(labels)
labels = LE.transform(labels)
# Labels have been replaced with integers.
np.unique(labels)

In [ ]:
#binary
for i in range(len(labels)):
 if labels[i]==0:
   labels[i]=0
 else :
   labels[i]=1
np.unique(labels)
labels=pd.get_dummies(labels)
labels = scaler.fit_transform(labels)
labels[0]

**Splitting the data**
Final step to data preparation is splitting the data into traning and testing sets. For this there already exists sklearn function that does all the splitting for us. This step is important so we can have representative data for evaluating our model. Both train and test samples should contain similar data variance.

In [ ]:
from sklearn.model_selection import train_test_split
# The next step is to split training and testing data. For this we will use sklearn function train_test_split().
# In the first step we will split the data in training and remaining dataset
x_train, x_rem, y_train, y_rem = train_test_split(features,labels, train_size=0.8)
x_train.shape, x_rem.shape, y_train.shape, y_rem.shape

In [ ]:
x_valid, x_test, y_valid, y_test = train_test_split(x_rem,y_rem, test_size=0.5)
x_valid.shape,x_test.shape, y_valid.shape, y_test.shape

In [ ]:
#instancier le modéle
from tensorflow.keras.models import Sequential
model=Sequential()
#Créer la couche entrée totalement connecté avec la couche dense
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, AveragePooling1D,BatchNormalization,SpatialDropout1D,MaxPool1D
from tensorflow.keras.layers import LeakyReLU

In [ ]:
model.add(Conv1D(32,kernel_size =5,padding="same",input_shape =(x_train.shape[1],1)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(AveragePooling1D(pool_size=2))

# #2em block
model.add(Conv1D(64,kernel_size =5,padding="same",))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(AveragePooling1D(pool_size=2))
model.add(SpatialDropout1D(0.2))

#3em block
model.add(Conv1D(128,kernel_size =5,padding="same",))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(AveragePooling1D(pool_size=2))

#4em block
model.add(Conv1D(256,kernel_size =5,padding="same",))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(AveragePooling1D(pool_size=2))
model.add(SpatialDropout1D(0.2))


model.add(Flatten())
model.add(Dense(units=20, activation="relu"))

#model.add(Dropout(0.3))
#model.add(LeakyReLU(alpha=0.1))
model.add(Dense(2,activation="sigmoid"))
model.summary() #sert afficher résumés

In [ ]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy',keras.metrics.Precision()])

In [ ]:
#model.fit(x_train, y_train, epochs = 100, batch_size =1000)
#model.fit(x_train, y_train, epochs =80,batch_size =128,validation_data=(x_valid, y_valid),callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1))
from keras.callbacks import EarlyStopping
#history = model.fit(x_train, y_train, epochs =80,batch_size =128,validation_data=(x_valid, y_valid),callbacks = [EarlyStopping(monitor='val_loss', patience=5)])
history = model.fit(x_train, y_train, epochs =100,batch_size =5000,validation_data=(x_valid, y_valid))


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

loss_val = history.history['val_loss']
acc_val = history.history['loss']
epochs = range(1,76)
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.plot(epochs, acc_val, 'g', label='train loss')
plt.title('Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc_val = history.history['val_accuracy']
acc_train = history.history['accuracy']
epochs = range(1,101)
plt.plot(epochs, acc_val, 'b', label='validation acc')
plt.plot(epochs, acc_train, 'g', label='train_accuracy')
plt.title('accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#check accuaracy of prediction
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score, recall_score
#model.score(x_valid,y_valid)

In [ ]:
#cf_matrix=confusion_matrix(y_valid, y_pred)
y_test_arg=np.argmax(y_test1,axis=1)
Y_pred = np.argmax(model.predict(x_test1),axis=1)
print('Confusion Matrix')
cf_matrix=confusion_matrix(y_test_arg, Y_pred)
print(cf_matrix)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(cf_matrix, cmap='coolwarm',annot=True, linewidth=1,fmt="d")
plt.show()

In [ ]:
accuracy =accuracy_score(y_test_arg, Y_pred)*100
print(accuracy)

In [ ]:
recall = recall_score(y_test_arg, Y_pred , average="binary")
precision = precision_score(y_test_arg, Y_pred, average="binary")
FPR = cf_matrix[0][1]/(cf_matrix[0][1]+cf_matrix[1][1]) 

In [ ]:
print("Precision : " , precision*100)
print("Recall : ", recall*100)
print("Accuracy : ",accuracy)
print("FPR : ",FPR*100)